In [3]:
import mysql.connector as sql
import pandas as pd

In [4]:
con = sql.connect(host='localhost', database='shakespeare', user='root', password='ty970323')

In [5]:
df = pd.read_sql('SELECT * FROM Paragraphs', con=con)

In [6]:
df.head()

,WorkID,ParagraphID,ParagraphNum,CharID,PlainText,PhoneticText,StemText,ParagraphType,Section,Chapter,CharCount,WordCount
0,allswell,859862,1,xxx,"Enter BERTRAM, the COUNTESS of Rousillon, HELE...",ENTR BRTRM 0 KNTS OF RSLN HLN ANT LF AL IN BLK,enter bertram the countess of rousillon helen...,b,1,1,79,12
1,allswell,859863,3,Countess-aw,"In delivering my son from me, I bury a second ...",IN TLFRNK M SN FRM M I BR A SKNT HSBNT,in deliv my son from me i buri a second husband,b,1,1,55,11
2,allswell,859864,4,BERTRAM,"And I in going, madam, weep o'er my father's d...",ANT I IN KNK MTM WP OR M F0RS T0 AN BT I MST ...,and i in go madam weep oer my father death an...,b,1,1,154,28
3,allswell,859865,7,LAFEU,"You shall find of the king a husband, madam; y...",Y XL FNT OF 0 KNK A HSBNT MTM Y SR A F0R H 0T...,you shall find of the king a husband madam yo...,b,1,1,262,48
4,allswell,859866,12,Countess-aw,What hope is there of his majesty's amendment?\n,HT HP IS 0R OF HS MJSTS AMNTMNT,what hope i there of hi majesti amend,b,1,1,47,8


In [7]:
df.shape

(35629, 12)

In [8]:
def process_line(df, i, file):       # Write the character and text to the file
    character = df['CharID'][i]
    text = df['PlainText'][i]
    file.write(character + ':' + '\n')
    text = text.replace('[p]', '')
    if character == 'xxx':
        while text[0] == ' ':
            text = text[1:len(text)]
        if text[0] != '[':
            text = text.replace('[', '(')
            text = text.replace(']', ')')
            text = '[' + text
            text = text[0:len(text) - 2] + ']\n'
    file.write(text + '\n')
    file.write('\n')

In [9]:
prev_play = ''
prev_chapter = 1
prev_section = 1
cur_file = None
for i in range(len(df.WorkID)):
    
    if df.WorkID[i] != prev_play:
        if i > 0:
            cur_file.close()
        cur_file = open(df.WorkID[i] + '.txt', 'w')
        prev_play = df.WorkID[i]
        prev_chapter = 1
        prev_section = 1
        cur_file.write('Act 1, Scene 1\n\n')    # new play
    elif df.Section[i] != prev_section:
        cur_file.write('****************************************\n\n\n')   # Act change
        cur_file.write('Act ' + str(df.Section[i]) + ', Scene ' + str(df.Chapter[i]) + '\n\n')
        prev_chapter = df.Chapter[i]
        prev_section = df.Section[i]
    elif df.Chapter[i] != prev_chapter:
        cur_file.write('----------------------------------------\n' + '\n')   # Section change
        cur_file.write('Act ' + str(df.Section[i]) + ', Scene ' + str(df.Chapter[i]) + '\n\n')
        prev_chapter = df.Chapter[i]

    
    process_line(df, i, cur_file)

    if i == len(df.WorkID) - 1:
        cur_file.close()